# Getting started with our ultimate beginner guide!

## This tutorial will walk you through the basics of using the `usb` lighting package. Let's get started by training a FixMatch model on CIFAR-10!

In [1]:
from semilearn import get_dataset, get_data_loader, get_net_builder, get_algorithm, get_config, Trainer

/home/haoc/TorchSSL_Benchmark/Semi-supervised-learning-Hook/semilearn/nets/vit/vit.py:36: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(H == self.img_size[0], f"Input image height ({H}) doesn't match model ({self.img_size[0]}).")
/home/haoc/TorchSSL_Benchmark/Semi-supervised-learning-Hook/semilearn/nets/vit/vit.py:37: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(W == self.img_size[1], f"Input image width ({W}) doesn't match model ({self.img_size[1]}).")


: 

: 

## Step 1: define configs and create config

In [2]:
config = {
    'algorithm': 'fixmatch',
    'net': 'wrn_28_2',
    'use_pretrain': False,  # todo: add pretrain
    'pretrain_path': None,

    # optimization configs
    'epoch': 3,
    'num_train_iter': 150,
    'num_eval_iter': 50,
    'optim': 'SGD',
    'lr': 0.03,
    'momentum': 0.9,
    'batch_size': 64,
    'eval_batch_size': 64,

    # dataset configs
    'dataset': 'cifar10',
    'num_labels': 40,
    'num_classes': 10,
    'img_size': 32,
    'crop_ratio': 0.875,
    'data_dir': './data',

    # algorithm specific configs
    'hard_label': True,
    'uratio': 3,
    'ulb_loss_ratio': 1.0,

    # device configs
    'gpu': 0,
    'world_size': 1,
    'distributed': False,
}
config = get_config(config)

/bin/sh: 1: netstat: not found


## Step 2: create model and specify algorithm

In [3]:
algorithm = get_algorithm(config,  get_net_builder(config.net, from_name=False), tb_log=None, logger=None)

## Step 3: create dataset

In [4]:
dataset_dict = get_dataset(config, config.algorithm, config.dataset, config.num_labels, config.num_classes, data_dir=config.data_dir)
train_lb_loader = get_data_loader(config, dataset_dict['train_lb'], config.batch_size)
train_ulb_loader = get_data_loader(config, dataset_dict['train_ulb'], int(config.batch_size * config.uratio))
eval_loader = get_data_loader(config, dataset_dict['eval'], config.eval_batch_size)

Files already downloaded and verified
Files already downloaded and verified


## Step 4: train

In [5]:
trainer = Trainer(config, algorithm)
trainer.fit(train_lb_loader, train_ulb_loader, eval_loader)

Epoch: 0


/home/haoc/miniconda3/envs/usb/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[2022-08-21 18:19:28,969 INFO] confusion matrix
[2022-08-21 18:19:28,970 INFO] [[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]
[2022-08-21 18:19:28,973 INFO] evaluation metric
[2022-08-21 18:19:28,973 INFO] acc: 0.0953
[2022-08-21 18:19:28,974 INFO] precision: 0.0095
[2022-08-21 18:19:28,974 INFO] recall: 0.1000
[2022-08-21 18:19:28,975 INFO] f1: 0.0174


model saved: ./saved_models/fixmatch_cifar10/latest_model.pth
model saved: ./saved_models/fixmatch_cifar10/model_best.pth
Epoch: 1


/home/haoc/miniconda3/envs/usb/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[2022-08-21 18:19:38,818 INFO] confusion matrix
[2022-08-21 18:19:38,820 INFO] [[0.         0.75079872 0.         0.         0.24920128 0.
  0.         0.         0.         0.        ]
 [0.         0.87155963 0.         0.         0.12844037 0.
  0.         0.         0.         0.        ]
 [0.         0.86666667 0.         0.         0.13333333 0.
  0.         0.         0.         0.        ]
 [0.         0.90634441 0.         0.         0.09365559 0.
  0.         0.         0.         0.        ]
 [0.         0.9147541  0.         0.         0.0852459  0.
  0.         0.         0.         0.        ]
 [0.         0.88178914 0.         0.         0.11821086 0.
  0.    

model saved: ./saved_models/fixmatch_cifar10/latest_model.pth
model saved: ./saved_models/fixmatch_cifar10/model_best.pth
Epoch: 2


/home/haoc/miniconda3/envs/usb/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[2022-08-21 18:19:49,039 INFO] confusion matrix
[2022-08-21 18:19:49,040 INFO] [[0.         0.84025559 0.         0.         0.15974441 0.
  0.         0.         0.         0.        ]
 [0.         0.91743119 0.         0.         0.08256881 0.
  0.         0.         0.         0.        ]
 [0.         0.92698413 0.         0.         0.07301587 0.
  0.         0.         0.         0.        ]
 [0.         0.95166163 0.         0.         0.04833837 0.
  0.         0.         0.         0.        ]
 [0.         0.93770492 0.         0.         0.06229508 0.
  0.         0.         0.         0.        ]
 [0.         0.90415335 0.         0.         0.09584665 0.
  0.    

model saved: ./saved_models/fixmatch_cifar10/latest_model.pth
model saved: ./saved_models/fixmatch_cifar10/model_best.pth


## Step 5: evaluate

In [6]:
trainer.evaluate(eval_loader)

/home/haoc/miniconda3/envs/usb/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[2022-08-21 18:21:23,656 INFO] confusion matrix
[2022-08-21 18:21:23,658 INFO] [[0.         0.84025559 0.         0.         0.15974441 0.
  0.         0.         0.         0.        ]
 [0.         0.91743119 0.         0.         0.08256881 0.
  0.         0.         0.         0.        ]
 [0.         0.92698413 0.         0.         0.07301587 0.
  0.         0.         0.         0.        ]
 [0.         0.95166163 0.         0.         0.04833837 0.
  0.         0.         0.         0.        ]
 [0.         0.93770492 0.         0.         0.06229508 0.
  0.         0.         0.         0.        ]
 [0.         0.90415335 0.         0.         0.09584665 0.
  0.    

{'acc': 0.0996875,
 'precision': 0.016786053719491927,
 'recall': 0.09797262746277637,
 'f1': 0.024902520800984422}

## Step 6: predict

In [7]:
y_pred, y_logits = trainer.predict(eval_loader)